<a href="https://colab.research.google.com/github/BonecoClark/Trabalho-Backend/blob/main/Trabalho_de_Back_End.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%writefile init_db.sql
CREATE TABLE IF NOT EXISTS tarefas (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    titulo TEXT NOT NULL,
    descricao TEXT,
    status TEXT NOT NULL DEFAULT 'pendente',
    criado_em TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

Overwriting init_db.sql


In [3]:
import sqlite3
conn = sqlite3.connect('tarefas.db')
conn.execute('''
    CREATE TABLE IF NOT EXISTS tarefas (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        titulo TEXT NOT NULL,
        descricao TEXT,
        status TEXT NOT NULL DEFAULT 'pendente',
        criado_em TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
''')
conn.close()
print("✅ Banco de dados inicializado.")

✅ Banco de dados inicializado.


In [4]:
%%writefile server.py
<COLE AQUI TODO O CÓDIGO DO server.py ACIMA>

Overwriting server.py


In [5]:
%%writefile server.py

import json
import sqlite3
from http.server import BaseHTTPRequestHandler, HTTPServer
from urllib.parse import urlparse, parse_qs
import re
from datetime import datetime

DB_PATH = 'tarefas.db'

def init_db():
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS tarefas (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            titulo TEXT NOT NULL,
            descricao TEXT,
            status TEXT NOT NULL DEFAULT 'pendente',
            criado_em TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    conn.commit()
    conn.close()

def get_db_connection():
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row  # Permite acessar colunas por nome
    return conn

class TaskHandler(BaseHTTPRequestHandler):
    def _send_response(self, data, status=200):
        self.send_response(status)
        self.send_header('Content-type', 'application/json')
        self.send_header('Access-Control-Allow-Origin', '*')  # Útil para testes
        self.end_headers()
        self.wfile.write(json.dumps(data, default=str).encode('utf-8'))

    def do_GET(self):
        parsed_path = urlparse(self.path)
        path = parsed_path.path

        if path == '/tasks':
            conn = get_db_connection()
            tarefas = conn.execute('SELECT * FROM tarefas').fetchall()
            conn.close()
            tarefas_list = [dict(t) for t in tarefas]
            self._send_response(tarefas_list)

        elif re.match(r'^/tasks/\d+$', path):
            task_id = int(path.split('/')[-1])
            conn = get_db_connection()
            tarefa = conn.execute('SELECT * FROM tarefas WHERE id = ?', (task_id,)).fetchone()
            conn.close()
            if tarefa:
                self._send_response(dict(tarefa))
            else:
                self._send_response({'error': 'Tarefa não encontrada'}, 404)
        else:
            self._send_response({'error': 'Rota não encontrada'}, 404)

    def do_POST(self):
        if self.path == '/tasks':
            content_length = int(self.headers['Content-Length'])
            post_data = self.rfile.read(content_length)
            try:
                data = json.loads(post_data)
                titulo = data.get('titulo')
                descricao = data.get('descricao', '')
                status = data.get('status', 'pendente')

                if not titulo:
                    self._send_response({'error': 'Título é obrigatório'}, 400)
                    return

                conn = get_db_connection()
                cursor = conn.cursor()
                cursor.execute('''
                    INSERT INTO tarefas (titulo, descricao, status)
                    VALUES (?, ?, ?)
                ''', (titulo, descricao, status))
                conn.commit()
                new_id = cursor.lastrowid
                conn.close()

                self._send_response({'id': new_id, 'titulo': titulo, 'descricao': descricao, 'status': status, 'criado_em': datetime.now().isoformat()}, 201)
            except json.JSONDecodeError:
                self._send_response({'error': 'JSON inválido'}, 400)
        else:
            self._send_response({'error': 'Rota não permitida'}, 405)

    def do_PUT(self):
        if re.match(r'^/tasks/\d+$', self.path):
            task_id = int(self.path.split('/')[-1])
            content_length = int(self.headers['Content-Length'])
            put_data = self.rfile.read(content_length)
            try:
                data = json.loads(put_data)
                titulo = data.get('titulo')
                descricao = data.get('descricao')
                status = data.get('status')

                conn = get_db_connection()
                # Verifica se a tarefa existe
                tarefa = conn.execute('SELECT * FROM tarefas WHERE id = ?', (task_id,)).fetchone()
                if not tarefa:
                    conn.close()
                    self._send_response({'error': 'Tarefa não encontrada'}, 404)
                    return

                # Atualiza apenas os campos fornecidos
                updates = []
                params = []
                if titulo is not None:
                    updates.append('titulo = ?')
                    params.append(titulo)
                if descricao is not None:
                    updates.append('descricao = ?')
                    params.append(descricao)
                if status is not None:
                    updates.append('status = ?')
                    params.append(status)

                if not updates:
                    conn.close()
                    self._send_response({'error': 'Nenhum campo para atualizar'}, 400)
                    return

                params.append(task_id)
                query = f"UPDATE tarefas SET {', '.join(updates)} WHERE id = ?"
                conn.execute(query, params)
                conn.commit()
                conn.close()

                # Retorna a tarefa atualizada
                conn = get_db_connection()
                updated = conn.execute('SELECT * FROM tarefas WHERE id = ?', (task_id,)).fetchone()
                conn.close()
                self._send_response(dict(updated))
            except json.JSONDecodeError:
                self._send_response({'error': 'JSON inválido'}, 400)
        else:
            self._send_response({'error': 'Rota inválida'}, 404)

    def do_DELETE(self):
        if re.match(r'^/tasks/\d+$', self.path):
            task_id = int(self.path.split('/')[-1])
            conn = get_db_connection()
            tarefa = conn.execute('SELECT * FROM tarefas WHERE id = ?', (task_id,)).fetchone()
            if not tarefa:
                conn.close()
                self._send_response({'error': 'Tarefa não encontrada'}, 404)
                return

            conn.execute('DELETE FROM tarefas WHERE id = ?', (task_id,))
            conn.commit()
            conn.close()
            self._send_response({'message': 'Tarefa deletada com sucesso'}, 200)
        else:
            self._send_response({'error': 'Rota inválida'}, 404)

    # Permitir CORS para testes (opcional)
    def do_OPTIONS(self):
        self.send_response(200)
        self.send_header('Access-Control-Allow-Origin', '*')
        self.send_header('Access-Control-Allow-Methods', 'GET, POST, PUT, DELETE, OPTIONS')
        self.send_header('Access-Control-Allow-Headers', 'Content-Type')
        self.end_headers()

def run_server(port=8000):
    init_db()
    server_address = ('', port)
    httpd = HTTPServer(server_address, TaskHandler)
    print(f"Servidor rodando na porta {port}...")
    httpd.serve_forever()

if __name__ == '__main__':
    run_server()

Overwriting server.py


In [6]:
# Célula 2: Rodar servidor em segundo plano
import threading
import time

# Importar o módulo server.py (não executa main, só carrega as funções)
import server

# Iniciar servidor em thread
server_thread = threading.Thread(target=server.run_server, args=(8000,), daemon=True)
server_thread.start()
time.sleep(1)  # Esperar o servidor iniciar
print("✅ Servidor iniciado em segundo plano na porta 8000.")

Servidor rodando na porta 8000...
✅ Servidor iniciado em segundo plano na porta 8000.


In [7]:
%%writefile client.py

import requests
import sys
import json

BASE_URL = 'http://localhost:8000/tasks'

def criar_tarefa(titulo, descricao='', status='pendente'):
    payload = {'titulo': titulo, 'descricao': descricao, 'status': status}
    try:
        r = requests.post(BASE_URL, json=payload)
        if r.status_code == 201:
            print("✅ Tarefa criada com sucesso!")
            print(r.json())
        else:
            print(f"❌ Erro ao criar tarefa: {r.status_code} - {r.json().get('error', 'Erro desconhecido')}")
    except requests.exceptions.ConnectionError:
        print("❌ Não foi possível conectar ao servidor. Verifique se ele está rodando.")

def listar_tarefas():
    try:
        r = requests.get(BASE_URL)
        if r.status_code == 200:
            tarefas = r.json()
            if tarefas:
                for t in tarefas:
                    print(f"[{t['id']}] {t['titulo']} - {t['status']} ({t['criado_em']})")
                    if t['descricao']:
                        print(f"    Descrição: {t['descricao']}")
            else:
                print("📝 Nenhuma tarefa encontrada.")
        else:
            print(f"❌ Erro ao listar tarefas: {r.status_code}")
    except requests.exceptions.ConnectionError:
        print("❌ Não foi possível conectar ao servidor.")

def ver_tarefa(task_id):
    try:
        r = requests.get(f"{BASE_URL}/{task_id}")
        if r.status_code == 200:
            t = r.json()
            print(f"📌 Tarefa {t['id']}: {t['titulo']}")
            print(f"   Status: {t['status']}")
            print(f"   Criada em: {t['criado_em']}")
            if t['descricao']:
                print(f"   Descrição: {t['descricao']}")
        elif r.status_code == 404:
            print("❌ Tarefa não encontrada.")
        else:
            print(f"❌ Erro: {r.status_code}")
    except requests.exceptions.ConnectionError:
        print("❌ Não foi possível conectar ao servidor.")

def atualizar_tarefa(task_id, status=None, titulo=None, descricao=None):
    payload = {}
    if status is not None:
        payload['status'] = status
    if titulo is not None:
        payload['titulo'] = titulo
    if descricao is not None:
        payload['descricao'] = descricao

    if not payload:
        print("⚠️ Nada para atualizar.")
        return

    try:
        r = requests.put(f"{BASE_URL}/{task_id}", json=payload)
        if r.status_code == 200:
            print("✅ Tarefa atualizada com sucesso!")
            print(r.json())
        elif r.status_code == 404:
            print("❌ Tarefa não encontrada.")
        else:
            print(f"❌ Erro ao atualizar: {r.status_code} - {r.json().get('error', '')}")
    except requests.exceptions.ConnectionError:
        print("❌ Não foi possível conectar ao servidor.")

def deletar_tarefa(task_id):
    try:
        r = requests.delete(f"{BASE_URL}/{task_id}")
        if r.status_code == 200:
            print("🗑️ Tarefa deletada com sucesso!")
        elif r.status_code == 404:
            print("❌ Tarefa não encontrada.")
        else:
            print(f"❌ Erro ao deletar: {r.status_code}")
    except requests.exceptions.ConnectionError:
        print("❌ Não foi possível conectar ao servidor.")

def main():
    if len(sys.argv) < 2:
        print("Uso: python client.py <comando> [argumentos]")
        print("Comandos:")
        print("  criar <titulo> [descricao] [status]")
        print("  listar")
        print("  ver <id>")
        print("  atualizar <id> [--status <status>] [--titulo <titulo>] [--descricao <descricao>]")
        print("  deletar <id>")
        return

    comando = sys.argv[1]

    if comando == 'criar':
        if len(sys.argv) < 3:
            print("Erro: título é obrigatório.")
            return
        titulo = sys.argv[2]
        descricao = sys.argv[3] if len(sys.argv) > 3 else ''
        status = sys.argv[4] if len(sys.argv) > 4 else 'pendente'
        criar_tarefa(titulo, descricao, status)

    elif comando == 'listar':
        listar_tarefas()

    elif comando == 'ver':
        if len(sys.argv) < 3:
            print("Erro: ID da tarefa é obrigatório.")
            return
        try:
            task_id = int(sys.argv[2])
            ver_tarefa(task_id)
        except ValueError:
            print("Erro: ID deve ser um número inteiro.")

    elif comando == 'atualizar':
        if len(sys.argv) < 3:
            print("Erro: ID da tarefa é obrigatório.")
            return
        try:
            task_id = int(sys.argv[2])
        except ValueError:
            print("Erro: ID deve ser um número inteiro.")
            return

        # Parser simples de argumentos nomeados
        args = sys.argv[3:]
        kwargs = {}
        i = 0
        while i < len(args):
            if args[i] == '--status' and i + 1 < len(args):
                kwargs['status'] = args[i+1]
                i += 2
            elif args[i] == '--titulo' and i + 1 < len(args):
                kwargs['titulo'] = args[i+1]
                i += 2
            elif args[i] == '--descricao' and i + 1 < len(args):
                kwargs['descricao'] = args[i+1]
                i += 2
            else:
                print(f"⚠️ Argumento desconhecido ignorado: {args[i]}")
                i += 1

        if not kwargs:
            print("⚠️ Nenhum campo válido para atualizar.")
            return

        atualizar_tarefa(task_id, **kwargs)

    elif comando == 'deletar':
        if len(sys.argv) < 3:
            print("Erro: ID da tarefa é obrigatório.")
            return
        try:
            task_id = int(sys.argv[2])
            deletar_tarefa(task_id)
        except ValueError:
            print("Erro: ID deve ser um número inteiro.")
    else:
        print(f"Comando desconhecido: {comando}")

if __name__ == '__main__':
    main()

Overwriting client.py


In [8]:
!pip install requests -q

In [9]:
!python client.py criar "Clark" "Emily" pendente
!python client.py listar
!python client.py ver 1
!python client.py atualizar 1 --status completo
!python client.py deletar 1

✅ Tarefa criada com sucesso!
{'id': 8, 'titulo': 'Clark', 'descricao': 'Emily', 'status': 'pendente', 'criado_em': '2025-10-01T14:00:21.475611'}


127.0.0.1 - - [01/Oct/2025 14:00:21] "POST /tasks HTTP/1.1" 201 -


[2] emily - pendente (2025-10-01 13:40:09)
    Descrição: cliente
[3] Clark - pendente (2025-10-01 13:46:19)
    Descrição: Emily
[4] Clark - pendente (2025-10-01 13:48:34)
    Descrição: Emily
[5] Clark - pendente (2025-10-01 13:53:49)
    Descrição: Emily
[6] Clark - pendente (2025-10-01 13:55:19)
    Descrição: Emily
[7] Clark - pendente (2025-10-01 13:55:24)
    Descrição: Emily
[8] Clark - pendente (2025-10-01 14:00:21)
    Descrição: Emily


127.0.0.1 - - [01/Oct/2025 14:00:21] "GET /tasks HTTP/1.1" 200 -


❌ Tarefa não encontrada.


127.0.0.1 - - [01/Oct/2025 14:00:22] "GET /tasks/1 HTTP/1.1" 404 -


❌ Tarefa não encontrada.


127.0.0.1 - - [01/Oct/2025 14:00:22] "PUT /tasks/1 HTTP/1.1" 404 -


❌ Tarefa não encontrada.


127.0.0.1 - - [01/Oct/2025 14:00:22] "DELETE /tasks/1 HTTP/1.1" 404 -


In [9]:
!ls -la

total 52
drwxr-xr-x 1 root root  4096 Oct  1 13:48 .
drwxr-xr-x 1 root root  4096 Oct  1 13:08 ..
-rw-r--r-- 1 root root  5879 Oct  1 13:48 client.py
drwxr-xr-x 4 root root  4096 Sep 29 13:36 .config
-rw-r--r-- 1 root root   223 Oct  1 13:48 init_db.sql
drwxr-xr-x 2 root root  4096 Oct  1 13:48 __pycache__
drwxr-xr-x 1 root root  4096 Sep 29 13:37 sample_data
-rw-r--r-- 1 root root  6760 Oct  1 13:48 server.py
-rw-r--r-- 1 root root 12288 Oct  1 13:48 tarefas.db


In [1]:
!kill -9 $(lsof -t -i:8000) 2>/dev/null || echo "Nenhum processo na porta 8000"

Nenhum processo na porta 8000
